In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import shap
import matplotlib.pyplot as plt

# Comando "mágico" para exibir gráficos do Matplotlib inline
%matplotlib inline

# ESSENCIAL: Carrega o Javascript do SHAP no notebook
# Isso fará o force_plot (o .html) funcionar
shap.initjs()

In [ ]:
# ============================
# 1. Gerar dados simulados
# ============================
rng = np.random.default_rng(42)
n = 200
renda = rng.integers(1000, 10000, n)
historico = rng.integers(0, 10, n)
idade = rng.integers(18, 70, n)

# Regra simulada: renda e histórico altos → aprovado
aprovado = ((renda > 5000) & (historico > 5)).astype(int)

df = pd.DataFrame({
    "renda": renda,
    "historico": historico,
    "idade": idade,
    "aprovado": aprovado
})

# ============================
# 2. Treinar o modelo
# ============================
X = df[["renda", "historico", "idade"]]
y = df["aprovado"]

modelo = DecisionTreeClassifier(max_depth=3, random_state=42)
modelo.fit(X, y)

In [ ]:
# ============================
# 3. Explicabilidade com SHAP
# ============================
explainer = shap.Explainer(modelo, X)
shap_values = explainer(X)

print("Cálculo do SHAP concluído.")

In [ ]:
# --- Importância global (summary plot) para APROVADO
print("Importância Global - Classe 1 (Aprovado)")

# O gráfico será exibido automaticamente abaixo desta célula
shap.summary_plot(shap_values[..., 1], X)

In [ ]:
# --- Importância global (summary plot) para REPROVADO
print("Importância Global - Classe 0 (Reprovado)")

shap.summary_plot(shap_values[..., 0], X)

In [ ]:
# --- Explicação local (force plot)

sample_idx = 0
output_idx = 0 # Classe 0 (Reprovado)

# Usamos a mesma lógica do seu script para criar o gráfico
force_plot = shap.force_plot(
    float(explainer.expected_value[output_idx]),
    shap_values.values[sample_idx, :, output_idx],
    X.iloc[sample_idx, :]
)

# Ao colocar a variável como a última linha da célula,
# o Jupyter irá renderizá-la automaticamente (graças ao shap.initjs() da Célula 1)
force_plot